<a href="https://colab.research.google.com/github/andreacangiani/NSPDE-ANA2024/blob/main/Python/CP3_worked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finite Difference for 1D reaction-advection-diffusion problem

1. Finite Difference solver for the reaction-advection-diffusion problem with nonconstant coefficients:

$-\alpha(x) u''(x)+\beta(x) u'(x)+\gamma(x) u(x)=f(x) \quad \in (a,b)$

$u(a)=0, \quad u(b)=0$.

Implement FD method including the boundary conditions in the system.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

Function computing the 1D FD algebric system

In [2]:
def FD1D(omega,N,alpha,beta,gamma,rhs):
  # FD system matrix and rhs for
  # diffusion-advection-reaction problem with coeffs
  # alpha,beta,gamma,rhs
  # homogeneous Dirichlet bc
  # uniform grid in sparse CSR format

  # grid
  h = (omega[1]-omega[0])/N
  x =np.linspace(omega[0],omega[1],N+1)

  # compute coeffs and rhs values
  diff = alpha(x)
  conv = beta(x)
  reac = gamma(x)
  F = rhs(x)

  # assemble system matrix
  diff_term = [-diff[1:N+1], 2*diff, -diff[0:-1]]
  conv_term = [-conv[1:N+1], conv[0:-1]]

  A = (1./h**2) * sp.diags(diff_term,[-1,0,1],format="csr")+(1./(2*h)) * sp.diags(conv_term,[-1,1],format="csr") + sp.diags(reac,0,format="csr")

  # modify system to account for homogeneous Dirichlet boundary conditions
  A[0,0] = 1; A[0,1] = 0; F[0] = 0
  A[N,N] = 1; A[N,N-1] = 0; F[N] = 0

  return A, F

Solve the boundary value problem with:

$a=0,\, b=Π$

$\alpha(x)=1+\sin(x),\,  \beta(x)=1,\, \gamma(x)=x,\, f(x)=1$

In [ ]:
# Problem domain
omega =[0, np.pi]

# Problem data
alpha = lambda x : 1 + np.sin(x)
beta = lambda x : np.ones(len(x))
gamma = lambda x : x
rhs = lambda x : np.ones(len(x))

# Grid parameters
N=50
x =np.linspace(omega[0],omega[1],N+1)

# Assemble FD system
A, F = FD1D(omega,N,alpha,beta,gamma,rhs)

# solve
uh=sp.linalg.spsolve(A,F)

# plot solution
plt.plot(x,uh)


Compute experimental order of convergence (EOC) using knowledge that

$|| U-U_h ||_\infty \approx C h^k$

with $C$ independent of $h$. Hence,

$\frac{|| U-U_{h_1} ||_\infty}{|| U-U_{h_2} ||_\infty}\approx\large(\frac{h_1}{h_2}\large)^k$,

and then,

$k\approx\frac{\log || U-U_{h_1} ||_\infty-\log|| U-U_{h_2} ||_\infty}{\log h_1 - \log h_2}$.

Notice that to estimate the EOC you need to run at least two experiments, for instance with $h_1=h$, $h_2=h/2$.

For simplicity, let's take

$\alpha(x)=1,\,  \beta(x)=0,\, \gamma(x)=0$

and

$u=\sin(x)$ so that $f=\sin(x)$


In [ ]:
# Problem domain
omega =[0, np.pi]

# Problem data
alpha = lambda x : np.ones(len(x))
beta = lambda x : 0. * np.ones(len(x))
gamma = lambda x : 0. * np.ones(len(x))
rhs = lambda x : np.sin(x)
u = lambda x : np.sin(x)

# Number of experiments
no_exp = 20

# initialise with first experiment
# Grid parameters
N=5
x =np.linspace(omega[0],omega[1],N+1)

# Assemble FD system
A, F = FD1D(omega,N,alpha,beta,gamma,rhs)

# solve
uh=sp.linalg.spsolve(A,F)

# compute error
err1 = max(abs(u(x)-uh))
h1 = (omega[1]-omega[0])/N


# loop over remaining experiments
for i in range(no_exp-1):
  # fix the mesh
  N = 2 * N
  x =np.linspace(omega[0],omega[1],N+1)

  # evaluate system for given N
  A, F = FD1D(omega,N,alpha,beta,gamma,rhs)
  # Solve
  uh=sp.linalg.spsolve(A,F)
  # Compute error
  err2 = max(abs(u(x)-uh))
  h2 = (omega[1]-omega[0])/N

  print(err2)
  #print((np.log(err1)-np.log(err2))/(np.log(h1)-np.log(h2)))

  # Update
  err1=err2
  h1=h2



What if the exact solution is not known? Then we can still estiate the EOC but three experiments. Indeed, letting $h_2=\theta h_1$, we have

$|| U_{h_2}-U_{h_1} ||_\infty\le || U-U_{h_1} ||_\infty + || U-U_{h_2} ||_\infty\approx C h_1^k+C h_2^k \approx C (1+\theta^k) h_1^k$

Now, given also $h_3 = \theta h_2$, we have similarly

$|| U_{h_3}-U_{h_2} ||_\infty \approx C (1+\theta^k) h_2^k=C (1+\theta^k)\theta^k h_1^k$

hence,

$\frac{|| U_{h_2}-U_{h_1} ||_\infty}{|| U_{h_3}-U_{h_2} ||_\infty}\approx \frac{C (1+\theta^k) h_1^k}{C (1+\theta^k)\theta^k h_1^k}=\theta^{-k}$

from which $k$ can be estimated as before by passing to the logs.

NOTE! The discrete solutions are defined at different sets of points so the above comparison is to be intended on the set of common points!

**Exercise 1:** try this out for the problem with

$\alpha(x)=1+\sin(x),\,  \beta(x)=1,\, \gamma(x)=x, \, f=1$.

Check that you still get the expected rate of convergence.

**Exersie 2.** Modify the FD1D routine to solve problems with nonhomogeneous boundary conditions.

**Exersie 3.** Consider now the problem defined by:

$a=0$, $b=1$, $u(0)=1$, $u(1)=0$ $f=0$ and

$\alpha=\beta=\gamma=1$.

Solve this problem using a grid made of $N=10$ intervals and plot the solution.

Next, try with $\alpha=0.01$, $\beta=1$, $\gamma=0$. What happens? Can you explain? Experiment by taking finer grids.